# Grattage de verbes

Cet algorithme est utilisé pour extraire les verbes du site web du **Le conjuguer** du **Le Figaro**
adresse: https://leconjugueur.lefigaro.fr/frlistedeverbe.php

In [1]:
import requests
from bs4 import BeautifulSoup

In [64]:
url_racine = 'https://leconjugueur.lefigaro.fr'
url_liste_verbes = 'https://leconjugueur.lefigaro.fr/frlistedeverbe.php'
chemin_acces_fichier_verbes_sans_info = 'liste_verbes_sans_info.json'
chemin_acces_fichier_verbes_avec_info = 'liste_verbes_avec_info.json'
CODE_SUCCES = 200

In [26]:
def grattage(lien:str):
    res = requests.get(lien)
    if res.status_code == CODE_SUCCES:
        return res.status_code, BeautifulSoup(res.text, 'html.parser')
    return res.status_code, None 

In [27]:
code_succes, soup_data = grattage(url_liste_verbes)
if code_succes == CODE_SUCCES:
    verbes_par_lettre_de_debut = soup_data.find("div", {"id": "pop"})

## Verbe sans groupe et traduction en anglais

In [85]:
prec_lettre = ''
verbes_sans_info = {}
for element in verbes_par_lettre_de_debut.find_all():
    if element.name == 'h2':
        prec_lettre = element.text.split('en')[1].strip()
    elif element.name == 'p' and prec_lettre != '':
        verbes_sans_info[prec_lettre] = []
        for element_verbe in element.find_all('a'):
            verbes_sans_info[prec_lettre].append(
                {
                    'verbe': f"{element_verbe.text}".strip().lower(),
                    'url': f"{url_racine}{element_verbe['href']}",
                }
            )

In [92]:
import json

In [94]:
with open(chemin_acces_fichier_verbes_sans_info, 'w') as out_file:
    json.dump(verbes_sans_info, out_file, indent=4)

## Verbe avec groupe et traduction en anglais

In [98]:
from tqdm import tqdm
total = 0
for lettre in verbes_sans_info: 
  total +=  len(verbes_sans_info[lettre])

In [103]:
prec_lettre = ''
verbes_avec_info = {}

iteration = 0
pbar = tqdm(total=total)
for element in verbes_par_lettre_de_debut.find_all():
    if element.name == 'h2':
        prec_lettre = element.text.split('en')[1].strip()
    elif element.name == 'p' and prec_lettre != '':
        verbes_avec_info[prec_lettre] = []
        for element_verbe in element.find_all('a'):
            verbe = f"{element_verbe.text}".strip().lower()
            conjuguaison_url = f"{url_racine}{element_verbe['href']}"
            
            # Extraction du groupe et de la traduction du verbe en anglais
            groupe = None
            code_succes, res = grattage(conjuguaison_url)
            if code_succes == CODE_SUCCES and res != None:
                try:
                    
                    #Extraction du groupe
                    info = res.find("div", {"id": "verbeNav"}).find('p')
                    groupe = info.find('b').text.strip()
                    
                    #Extraction de la traduction en anglais
                    traduction = ''
                    for mot in info.text.split('anglaise :')[-1].split(' '):
                        if mot.strip().lower() == verbe:
                            break
                        traduction += mot + ' '
                except:
                    pass

            verbes_avec_info[prec_lettre].append(
                {
                    'verbe': verbe,
                    'url': conjuguaison_url,
                    'groupe': groupe,
                    'anglais': traduction.strip()
                }
            )
            pbar.set_description(f"Iteration {iteration + 1}")
            pbar.update(1)
            iteration +=1
pbar.close()

Iteration 1098:  56%|█████▌    | 1098/1976 [09:39<08:06,  1.80it/s]

In [104]:
with open(chemin_acces_fichier_verbes_avec_info, 'w') as out_file:
    json.dump(verbes_avec_info, out_file, indent=4)

## Version rapide avec multithreading 

Cette version rend le code exponentiellement plus rapide. Le pire des cas etant 8.5 fois plus rapide.

In [156]:
import time
from threading import Thread, RLock
from collections import defaultdict, deque

1. Grattage verbe sans groupe et traduction

In [ ]:
prec_lettre_d = ''
liste_verbes_sans_info = []
for element in verbes_par_lettre_de_debut.find_all():
    #Lettre de debut
    if element.name == 'h2':
        prec_lettre_d = element.text.split('en')[1].strip()
    elif element.name == 'p' and prec_lettre != '':
        #Liste de verbe selon la lettre de debut
        for element_verbe in element.find_all('a'):
            liste_verbes_sans_info.append(
                {
                    'verbe': f"{element_verbe.text}".strip().lower(),
                    'url': f"{url_racine}{element_verbe['href']}",
                    'lettre_debut': prec_lettre_d
                }
            )


2. Creation de packet de verbes de tailles de N avec le dernier packet avec une taille possibllement differente de N

In [148]:
ensembles_verbes = []
ensemble_courant = []
iteration = 0
trame = 50
liste_verbes_queue = deque(liste_verbes_sans_info)
while liste_verbes_queue:
    if iteration == trame:
        iteration = 0
        ensembles_verbes.append(ensemble_courant)
        ensemble_courant = []
    ensemble_courant.append(liste_verbes_queue.popleft())
    iteration += 1

if len(ensemble_courant) > 0: 
    ensembles_verbes.append(ensemble_courant)
    ensemble_courant = []

3. Definition de la fonction de grattage pour packet de verbes

In [211]:
def grattage_info_verbes(liste_verbes:list, resultat:list, id_thread=1):
    verbes_avec_info = defaultdict(list)
    for element in liste_verbes:
        verbe = element['verbe']
        conjuguaison_url = element['url']
        
        # Extraction du groupe et de la traduction du verbe en anglais
        groupe = None
        code_succes, res = grattage(conjuguaison_url)
        if code_succes == CODE_SUCCES and res != None:
            try:
                
                #Extraction du groupe
                info = res.find("div", {"id": "verbeNav"}).find('p')
                groupe = info.find('b').text.strip()
                
                #Extraction de la traduction en anglais
                traduction = ''
                for mot in info.text.split('anglaise :')[-1].split(' '):
                    if mot.strip().lower() == verbe:
                        break
                    traduction += mot + ' '
            except:
                pass

        verbes_avec_info[element['lettre_debut']].append(
            {
                'verbe': verbe,
                'url': conjuguaison_url,
                'groupe': groupe,
                'anglais': traduction.strip()
            }
        )
        
    with RLock():
        resultat.append(verbes_avec_info)
        print('Thread: ' ,id_thread, 'terminé')
    

In [213]:
liste_threads = []
verbes_avec_infos = []
nombre_threads = 0
for ensemble_verbes in (ensembles_verbes):
    thread = Thread(target=grattage_info_verbes, args=(ensemble_verbes, verbes_avec_infos, nombre_threads))
    thread.daemon = True
    thread.start()
    liste_threads.append(thread)
    nombre_threads += 1

while any([thread.is_alive() for thread in liste_threads]):
    time.sleep(1)

Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread: Thread:  0 terminé
 0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread: Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
 0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread: Thread:  0 terminé
 0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread:  0 terminé
Thread: Thread:  0 terminé
 0 terminé


In [ ]:
with open(chemin_acces_fichier_verbes_avec_info, 'w') as out_file:
    json.dump(verbes_avec_info, out_file, indent=4)